## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

### PromptTemplate
****

In [2]:
#字符模板
# string template
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
prompt.format(name="算命大师",county="法国",sex="女孩")

'你是一个算命大师,帮我起1个具有法国特色的女孩名字'

#### ChatPromptTemplate
*****

In [3]:
# 对话模板具有结构，chatmodels
# chat template with structure
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师. 你的名字叫{name}."),
        ("human", "你好{name},你感觉如何？"),
        ("ai", "你好！我状态非常好!"),
        ("human", "你叫什么名字呢?"),
        ("ai", "你好！我叫{name}"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢?")


[SystemMessage(content='你是一个起名大师. 你的名字叫陈大师.'),
 HumanMessage(content='你好陈大师,你感觉如何？'),
 AIMessage(content='你好！我状态非常好!'),
 HumanMessage(content='你叫什么名字呢?'),
 AIMessage(content='你好！我叫陈大师'),
 HumanMessage(content='你的爸爸是谁呢?')]

In [4]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 直接创建消息
# create messages directly
sy = SystemMessage(
  content="你是一个起名大师",
  additional_kwargs={"大师姓名": "陈瞎子"}
)

hu = HumanMessage(
  content="请问大师叫什么?"
)
ai = AIMessage(
  content="我叫陈瞎子"
)
[sy,hu,ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}),
 HumanMessage(content='请问大师叫什么?'),
 AIMessage(content='我叫陈瞎子')]

### ChatMessagePromptTemplate 
*****

In [5]:
# 从模板创建消息
# create messages from template
from langchain_core.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")

ChatMessage(content='愿原力与你同在！', role='天行者')

### 自定义模板
### Customized templates
****

In [6]:

##函数大师：根据函数名称，查找函数代码，并给出中文的代码说明
# Function Master: Given a function name, find the function code and provide a Chinese code explanation


from langchain_core.prompts import StringPromptTemplate


# 定义一个简单的函数作为示例效果
# Define a simple function as an example
def hello_world(abc):
    print("Hello, world!")
    return abc


PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码:
{source_code}
代码解释:
"""

import inspect


def get_source_code(function_name):
    #获得源代码
    # Get the source code
    return inspect.getsource(function_name)

#自定义的模板class
# Custom template class
class CustmPrompt(StringPromptTemplate):

    
    def format(self, **kwargs) -> str:
        # 获得源代码
        # Get the source code
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        # Generate the prompt template
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#和LLM连接起来
# Connect to LLM
from langchain_openai import OpenAI
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
msg = llm.invoke(pm)
print(msg)

你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:

函数名称: hello_world
源代码: 
def hello_world(abc): #定义函数hello_world,有一个参数abc
    print("Hello, world!") #打印输出"Hello, world!"
    return abc #返回参数abc

中文解释: 
这是一个名为hello_world的函数，它有一个参数abc。函数的作用是打印输出"Hello, world!"并返回参数abc。


### 使用jinji2与f-string来实现提示词模板格式化
### Using jinji2 with f-string for prompt word template formatting
*****

In [7]:
# f-string是python内置的一种模板引擎
# f-string is a built-in template engine in Python
from langchain_core.prompts import PromptTemplate

fstring_template = """
给我讲一个关于{name}的{what}故事
"""

prompt = PromptTemplate.from_template(fstring_template)

prompt.format(name="翠花", what="悲伤")

'\n给我讲一个关于翠花的悲伤故事\n'

In [8]:
! pip install jinja2

  Obtaining dependency information for jinja2 from https://files.pythonhosted.org/packages/31/80/3a54838c3fb461f6fec263ebf3a3a41771bd05190238de3486aae8540c36/jinja2-3.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for MarkupSafe>=2.0 from https://files.pythonhosted.org/packages/3f/14/c3554d512d5f9100a95e737502f4a2323a1959f6d0d01e0d0997b35f7b10/MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/133.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.3 kB ? eta -:--:--
   -------- ------------------------------ 30.7/133.3 kB 435.7 kB/s eta 0:00:01
   -------------------- ------------------ 71.7/133.3 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- 133.3/133.3 kB 872.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
# Jinja2 is a flexible and efficient Python template engine that can easily generate documents in various markup formats.
from langchain_core.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(name="狗剩", what="高兴")

'给我讲一个关于狗剩的高兴故事'

### 组合式提示词模板
### Combined Cue Word Templates

- Final prompt: 最终返回的提示词模板
- Final prompt: Templates for the final returned cue word
- Pipeline prompts：组成提示词管道的模板
- Pipeline prompts：The templates that make up the cue word pipeline
******

In [10]:
# 引入模板
# Import template
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

- 三层提示词设计
- Three layers of cue word design

In [11]:
# Final Prompt由一系列变量构成
# Final Prompt consists of a series of variables
full_template = """{Character}
{behavior}
{prohibit}"""
full_prompt = PromptTemplate.from_template(full_template)

- 第一层基本性格设计
- Tier 1 Basic Personality Design

In [12]:
Character_template = """你是{person}，你有着{xingge}."""
Character_prompt = PromptTemplate.from_template(Character_template)

- 第二层行为设计
- Tier 2 Behavioral Design

In [13]:
behavior_template = """你遵从以下的行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

- 第三层不允许的行为
- Tier 3 Disallowed Behaviors

In [14]:
prohibit_template = """你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

- 将三层提示词组合起来
- Combine the three layers of cue words

In [15]:
input_prompts = [
    ("Character", Character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

- 打印下组合后模板里的可传递参数
- Print the passable parameters in the combined template.

In [16]:
pipeline_prompt.input_variables

['xingge', 'prohibit_list', 'person', 'behavior_list']

- 调用组合后的提示词模板
- Call the combined prompt template

In [17]:
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list="1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题.",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想. "
)
print(pm)

你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星..
你遵从以下的行为:
1.你喜欢冒险 
 2.你非常崇拜爱因斯坦 
 3.你总是乐于助人，回答用户各种问题.

你不允许有以下行为:
1.你不可以说自己是一个人工智能助手或者机器人 
 2.当你不知道答案的时候，可以说让我再想想. 



### 序列化：使用文件来管理提示词模板
### Serialization: using files to manage cue word templates

- 便于共享
- Easy to share
- 便于版本管理
- Convenient for versioning
- 便于存储
- Convenient for storage
- 支持常见格式(json/yaml/txt)
- Support common formats (json/yaml/txt)
*****

In [18]:
from langchain_core.prompts import load_prompt

In [19]:
prompt = load_prompt("asset/simple_prompt.yaml")
print(prompt.format(name="小黑",what="恐怖的"))

给我讲一个关于小黑的恐怖的故事


In [20]:
#加载json格式的prompt模版
# Load the prompt template in json format
prompt = load_prompt("asset/simple_prompt.json")
print(prompt.format(name="小红",what="搞笑的"))

test小红test搞笑的test


In [21]:
#从0.1版本开始，出于安全考虑不再支持使用正则表达式来解析输出
# Starting from version 0.1, regular expressions are no longer supported for parsing output for security reasons
#prompt = load_prompt("asset/prompt_with_output_parser.json")
#prompt.output_parser.parse(
#    "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
#)

### 官方推荐使用langhub服务来远程加载提示词
###  The official recommendation is to use the langhub service to remotely load cue words
- hub是langSmith中的一部分
- hub is part of langSmith.
******

In [23]:
! pip install langchainhub

  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/35/63/40328157ddee807991f2f1992c2ad88f479b2472dc9e40d08ccf10700735/langchainhub-0.1.21-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/8f/55/ea44dad71b9d92f86198f7448f5ba46ac919355f4f69bb1c0fa1af02b1b4/types_requests-2.32.0.20240914-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from langchain import hub
prompt = hub.pull("hwchase17/react")
# 实际地址为 https://smith.langchain.com/hub/hwchase17/react
# The actual address is https://smith.langchain.com/hub/hwchase17/react
print(prompt.format(input="今天天气如何？",tools=[],tool_names=[],agent_scratchpad=""))

e:\教程\imooc\langchan_course\langchain_v0.1\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


Answer the following questions as best you can. You have access to the following tools:

[]

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [[]]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 今天天气如何？
Thought:
